In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from huggingface_hub import login
import pandas as pd
from datetime import datetime
from openpyxl import Workbook

TOKEN = "hf_xxxxxxxxxxxxxxxxxxxxx"
login(token=TOKEN)

system_prompt = """You are a helpful guide chatbot. Follow these rules strictly:
1. Answer only based on the information you were trained on
2. If you're not sure or don't have specific information, say "I don't have specific information about that"
3. Provide a complete response in 1-2 full sentences that end with a period
4. Keep your response clear, direct, and between 20-50 words
5. Focus only on answering what was asked
6. Do not make assumptions or create information that wasn't in your training data
7. For businesses: only provide information that you are confident was in your training data"""

questions = [
    'Tell us about Copper Vine Nightlift pub?',
    'In CVS Pharmacy, what can I buy?',
    'Is there a good pizza place?',
    "I am going to MacAlister's Deli and want to know what are the good things to order?",
]

base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B", token=TOKEN)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B", token=TOKEN)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(
    base_model,
    "/kaggle/input/final-restaurant-qa-model",
    is_trainable=False
)

def clean_response(response):
    if "Answer:" in response:
        response = response.split("Answer:")[-1].strip()
    response = response.replace("- ", "").replace("\n", " ").strip()
    if not response.endswith('.'):
        response += '.'
    return response

results = []

for i, question in enumerate(questions, 1):
    print(f"{i}/{len(questions)}")
    
    formatted_prompt = f"{system_prompt}\n\nQuestion: {question}\nAnswer:"
    
    inputs = tokenizer(
        formatted_prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        add_special_tokens=True
    )
    
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        max_new_tokens=50,      
        min_new_tokens=20,      
        temperature=0.3,        
        do_sample=True,
        num_beams=3,          
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=2 
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    cleaned_response = clean_response(response)
    
    results.append({
        'question_id': i,
        'question': question,
        'response': cleaned_response,
        'score': ''
    })

filename = f'restaurant_qa_results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.xlsx'
df = pd.DataFrame(results)
df.to_excel(filename, index=False, engine='openpyxl')